In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# Standard libraries

import math
import os
import sys

os.environ['TF_GPU_ALLOCATOR'] = 'cuda_malloc_async'


# Third-party libraries
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from tqdm import tqdm

# Scikit-learn
from sklearn.calibration import calibration_curve
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Keras
import keras
from keras import backend as K
from keras.callbacks import EarlyStopping
from keras.layers import Input, Dense, Conv1D, Dropout, Activation, Flatten
from keras.metrics import AUC
from keras.models import Model, Sequential
from keras.optimizers import Adam
from keras.utils import Sequence

# TensorFlow
import tensorflow as tf
from tensorflow.keras.layers import Input, Dropout, Dense, Flatten, LayerNormalization, MultiHeadAttention
from tensorflow.keras.models import Model

# Local imports
from models import Attia_et_al_CNN

from utils import split_train_val_test
from sklearn.metrics import roc_auc_score

# Environment variables
os.environ['CUDA_VISIBLE_DEVICES'] = '3'

In [ ]:
data_df = pd.read_pickle('data/arrythmia_dataset.pickle')
conditions = ['1AVB', 'AF', 'AFIB', 'APB', 'AQW', 'IDC',
              'LVH', 'LVQRSAL', 'RBBB', 'SR', 'ST',
              'STDD', 'STE', 'STTC', 'SVT', 'TWC',
              'TWO']

output_size = len(conditions)

In [ ]:
# Load the data
X, y = data_df['wf'].to_numpy(), data_df[conditions].to_numpy()
y = y.astype(float)
X = np.stack(X, axis=0)

del data_df

print(f"X shape: {X.shape}")
print(f"y shape: {y.shape}")
print(f"y mean: {y.mean()}")

In [ ]:
# Split into train and test
X_train, X_val, X_test, y_train, y_val, y_test = split_train_val_test(X, y, train_size=0.70, val_size=0.15)
del X, y

In [ ]:
def generator(X, y, batch_size=8):
    row_nums = np.arange(X.shape[0])
    np.random.shuffle(row_nums)
    for i in range(0, len(row_nums), batch_size):
        current_idxs = row_nums[i:i+batch_size]

        yield X[current_idxs], y[current_idxs,:]

output_signature = (
    tf.TensorSpec(shape=(None, 5000, 12), dtype=tf.float32),
    tf.TensorSpec(shape=(None, output_size), dtype=tf.float32)
)

train_ds = tf.data.Dataset.from_generator(generator=lambda: generator(X_train,y_train, 8), output_signature=output_signature)
val_ds = tf.data.Dataset.from_generator(generator=lambda: generator(X_val,y_val, 8), output_signature=output_signature)

In [ ]:
def train_single_model(hyperparameters, train_ds, val_ds):
    
    #create the model
    model = Attia_et_al_CNN(
        filter_numbers = hyperparameters['filter_numbers'], 
        kernel_widths = hyperparameters['kernel_widths'], 
        pool_sizes = hyperparameters['pool_sizes'], 
        spatial_num_filters = hyperparameters['spatial_num_filters'], 
        dense_dropout_rate = hyperparameters['dense_dropout_rate'], 
        spatial_dropout_rate = hyperparameters['spatial_dropout_rate'], 
        dense_units = hyperparameters['dense_units'], 
        use_spatial_layer = hyperparameters['use_spatial_layer'], 
        verbose = hyperparameters['verbose'], 
        output_size = output_size,
    ).build()
    
    #create optimizers and compile model
    learning_rate =1e-3
    reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss',  # Monitor validation loss
        factor=0.5,  # Reduce learning rate by half when triggered
        patience=3,  # Number of epochs with no improvement to trigger the callback
        verbose=1,  # Print messages
        min_lr=1e-8  # Minimum learning rate
    )
    early_stopping = EarlyStopping(monitor='val_loss', patience=6, mode='min', restore_best_weights=True)
    model.compile(loss='binary_crossentropy', optimizer=tf.keras.optimizers.legacy.Adam(learning_rate=learning_rate), metrics=['accuracy', AUC(name='auc')])
    # Training parameters
    EPOCHS = 50  # You can adjust based on your needs
    
    #fit the model
    history = model.fit(train_ds,
        epochs=EPOCHS,
        shuffle=True,
        validation_data=val_ds,
        callbacks=[reduce_lr, early_stopping],
        verbose=1)
    
    y_pred = model.predict(X_test)
    
    #create dictionary of roc auc curves
    roc_auc_scores_dict = {}
    
    #evaluate the model
    for i, condition in enumerate(conditions):
        auc = roc_auc_score(y_test[:,i], y_pred[:,i])
        roc_auc_scores_dict[condition] = auc
        
    #return the model hyperparameters and the roc_auc scores
    return hyperparameters, roc_auc_scores_dict

In [ ]:
hype1 = {
    'filter_numbers' : [16, 16, 32, 32, 64, 64],
    'kernel_widths' : [5, 5, 5, 3, 3, 3],
    'pool_sizes' : [2, 2, 4, 2, 2, 4],
    'spatial_num_filters' : 64,
    'dense_dropout_rate' : 0.2,
    'spatial_dropout_rate' : 0.2,
    'dense_units' : [64, 32],
    'use_spatial_layer' : False,
    'verbose' : 1,
    'output_size' : output_size,
}

hype2 = {
    'filter_numbers' : [16, 16, 32],
    'kernel_widths' : [5, 5, 5],
    'pool_sizes' : [2, 2, 4],
    'spatial_num_filters' : 64,
    'dense_dropout_rate' : 0.2,
    'spatial_dropout_rate' : 0.2,
    'dense_units' : [64, 32],
    'use_spatial_layer' : False,
    'verbose' : 1,
    'output_size' : output_size,
}

hyper_param_ls = [hype2, hype1]

In [ ]:
hyper_param_to_auc_roc = {}

for h in hyper_param_ls:
    hyperparameters, roc_auc_scores_dict = train_single_model(h, train_ds, val_ds)